# Assignment 3


- Student Name: Karina Jonina
- Student ID: c00278440

- Module Name: Programming for Data Scientists
- Module Code: PROGC5201
- Module Leader: Paul Barry

- Course Name: 		Master of Science in Data Science
- Course Code:		 CW_KCDAR_M Y5

- Due Date: 10th January 2022 9:00am

In [ ]:
import pandas as pd
import datetime
from os import listdir
from os.path import isfile, join
import glob
import re
from datetime import datetime

In [ ]:
# gettting the CSV from the website
url = 'https://opendata-geohive.hub.arcgis.com/datasets/d8eb52d56273413b84b0187a4e9117be_0.csv?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D.'
# url = 'COVID-19.csv'
df = pd.read_csv(url)


In [ ]:
# Ensuring the Date is just date, and not datetime
df['Date'] = pd.to_datetime(df['Date']).dt.date
df.head()

In [ ]:
# examining columns and rows
df.shape

In [ ]:
# checking type for each column
df.info()

In [ ]:
# examining the names of the columns
df.columns

In [ ]:
# Dropping irrelevant columns
df = df.drop(columns = ['StatisticsProfileDate', 'FID'])

## Examining location
Below exploration showed that there are no unique locations in the dataset and therefore, redundant. 

In [ ]:
df['X'].unique()

In [ ]:
df['Y'].unique()

## Examining date
Below exploration showed that there are are unique dates in the dataset. 

In [ ]:
df['Date'].unique()

In [ ]:
df.head()

## Melting 

In [ ]:
# Melting
id_vars = ["X","Y","Date"]

df = pd.melt(frame=df,id_vars=id_vars, var_name="Column", value_name="Cases")

df.head(10)


### Dropping Columns (X and Y)

In [ ]:
df = df[[
    "Date",
    "Column",
    "Cases"
]]

In [ ]:
df.head()

# Getting the final dataset

In [ ]:
tmp_df = df["Column"].str.extract("(\D+)(\d+)(\D{2})(\d{2})", expand=False)    

# Name columns
tmp_df.columns = ["Aged", "age_lower","to", "age_upper"]

# Create `age`column based on `age_lower` and `age_upper`
tmp_df["Age_Range"] = tmp_df["age_lower"] + "-" + tmp_df["age_upper"]

tmp_df.head(15)

In [ ]:
# Merge 
df = pd.concat([df, tmp_df], axis=1)

# Drop unnecessary columns and rows
df = df.drop(["age_lower","to","age_upper" ], axis=1)

# sorting data by date
df = df.sort_values(ascending = True, by = ["Date"])

# reseting the dataframe index
df = df.reset_index()


In [ ]:
df["Column"].unique()

In [ ]:
df = df [[
    "Date",
    "Column",
    "Age_Range",
    "Cases"
]]

In [ ]:
df.head(50)

### Dropping rows with NaN in cases

In [ ]:
df1 = df.copy()

In [ ]:
df1.head(20)

In [ ]:
df1 = df1[~df1['Cases'].isna()]
df1["Cases"] = df1["Cases"].astype(int)
# reseting the dataframe index
df1 = df1.reset_index(drop = True)

In [ ]:
df1.head()

In [ ]:
df1.shape

### Dealing with ['Column'] column

In [ ]:
# Testing the regular expression
line = 'HospitalisedAged45to54'
match = re.findall('([A-Z][a-z]+)', line)
new_line = match[0] + " " + match[1] 
print(new_line)

In [ ]:
column_list = []
for i in  df1["Column"]:
    match = re.findall('([A-Z][a-z]+)', i)
    if len(match) == 4:
        new_line = match[0] + ' ' + match[1] + ' ' + match[2] + ' ' + match[3]
    if len(match) == 3:
        new_line = match[0] + ' ' + match[1] + ' ' + match[2]
    elif len(match) == 2:
        new_line = match[0] + ' ' + match[1] 
    else:
        new_line = match[0]
    
    column_list.append(new_line)

# examining the list
print(column_list)  

In [ ]:
column_list = pd.Series(column_list)
column_list = column_list.reset_index()

In [ ]:
df1 = pd.merge(df1, column_list, left_index = True, right_index = True)

In [ ]:
df1 = df1.rename(columns={'Column': "old_column", 0 : "Column"})

In [ ]:
df1 = df1[[
    "Date",
    "old_column",
    "Column",
    "Age_Range",
    "Cases"
]]

In [ ]:
df1.head()

### Saving to a new csv file

In [ ]:
df1.to_csv(r"tidy_data_df.csv", index =  False)